In [ ]:
%%capture
!pip install azure-cognitiveservices-speech

In [ ]:
%%capture
!pip install youtube-dl

In [ ]:
%%capture
!youtube-dl https://www.youtube.com/watch?v=cfjdfaqWY3Y

In [ ]:
%%capture
!ffmpeg -i Cúla4\ Ar\ Scoil\ _\ Ábhar\ -\ Mata\ _\ Téama\ -\ Bia-cfjdfaqWY3Y.mkv -acodec pcm_s16le -ac 1 -ar 16000 cfjdfaqWY3Y.wav

In [ ]:
import IPython
IPython.display.Audio('/content/cfjdfaqWY3Y.wav')

In [6]:
import azure.cognitiveservices.speech as speechsdk

Use either Key1 or Key2 (on [Azure Portal](https://portal.azure.com/), in "Keys and Endpoints" from the menu on the left hand side of the screen).

In [41]:
# put your subscription key here
_SUBS=''

In [35]:
_LOC='westeurope'

In [63]:
speech_config = speechsdk.SpeechConfig(region=_LOC, subscription=_SUBS)


In [29]:
audio_input=speechsdk.audio.AudioConfig(filename='cfjdfaqWY3Y.wav')

In [50]:
speech_config.speech_recognition_language = 'ga-IE'
speech_config.request_word_level_timestamps()
speech_config.output_format = speechsdk.OutputFormat(1)
speech_config.endpoint_id='https://westeurope.api.cognitive.microsoft.com/sts/v1.0/issuetoken'

In [54]:
speech_config.set_property(speechsdk.PropertyId.Speech_LogFilename, "azure.log")

In [64]:
# https://github.com/Azure-Samples/cognitive-services-speech-sdk/blob/master/samples/python/console/speech_sample.py
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE.md file in the project root for full license information.
import time
def speech_recognize_continuous_from_file(speech_config, audio_config):
    """performs continuous speech recognition with input from an audio file"""
    speech_config = speech_config
    audio_config = audio_config

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language='ga-IE', audio_config=audio_config)
    

    done = False

    def stop_cb(evt):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    def cancelled(evt):
      result = evt.result
      cancellation_details = result.cancellation_details
      print("Speech Recognition canceled: {}".format(cancellation_details.reason))
      if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(cancelled)
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

In [ ]:
speech_recognize_continuous_from_file(speech_config, audio_input)

In [ ]:
 !curl -v -X POST "https://{_LOC}.api.cognitive.microsoft.com/sts/v1.0/issueToken" -H "Ocp-Apim-Subscription-Key: {_SUBS}" -H "Content-type: application/x-www-form-urlencoded" -H "Content-Length: 0"

In [45]:
_TOK=''

In [ ]:
!curl -v -X POST "https://{_LOC}.stt.speech.microsoft.com/speech/recognition/interactive/cognitiveservices/v1?language=ga-IE" -H "Authorization: Bearer {_TOK}" -H "Transfer-Encoding: chunked" -H "Content-type: audio/wav; codec=audio/pcm; samplerate=16000" --data-binary @cfjdfaqWY3Y.wav

In [40]:
import time
transcript_display_list = []
transcript_ITN_list = []
confidence_list = []
words = []
global done
done = False
def parse_azure_result(evt):
  import json
  response = json.loads(evt.result.json)
  transcript_display_list.append(response['DisplayText'])
  confidence_list_temp = [item.get('Confidence') for item in response['NBest']]
  max_confidence_index = confidence_list_temp.index(max(confidence_list_temp))
  confidence_list.append(response['NBest'][max_confidence_index]['Confidence'])
  transcript_ITN_list.append(response['NBest'][max_confidence_index]['ITN'])
  words.extend(response['NBest'][max_confidence_index]['Words'])
  logger.debug(evt)
    
# Service callback that stops continuous recognition upon receiving an event `evt`
def stop_cb(evt):
  print('CLOSING on {}'.format(evt))
  speech_recognizer.stop_continuous_recognition()
  done = True
    
  # Do something with the combined responses
  print(transcript_display_list)
  print(confidence_list)
  print(words)
    
    
# Connect callbacks to the events fired by the speech recognizer
speech_recognizer.recognizing.connect(lambda evt: logger.debug('RECOGNIZING: {}'.format(evt)))
speech_recognizer.recognized.connect(parse_azure_result)
speech_recognizer.session_started.connect(lambda evt: logger.debug('SESSION STARTED: {}'.format(evt)))
speech_recognizer.session_stopped.connect(lambda evt: logger.debug('SESSION STOPPED {}'.format(evt)))
speech_recognizer.canceled.connect(lambda evt: logger.debug('CANCELED {}'.format(evt)))
# stop continuous recognition on either session stopped or canceled events
speech_recognizer.session_stopped.connect(stop_cb)
speech_recognizer.canceled.connect(stop_cb)
    
# Start continuous speech recognition
print("Initiating speech to text")
speech_recognizer.start_continuous_recognition()
while not done:
  time.sleep(.5)

Initiating speech to text


RuntimeError: ignored